# Analisis de Egresos Hospitalarios

In [1]:
import importlib

import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

import lectura_archivos as fr
import calculo_metricas as aux

from functools import reduce

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

importlib.reload(fr)
importlib.reload(aux)


<module 'calculo_metricas' from 'c:\\Users\\ppizarro\\Desktop\\Javier Rojas\\Javier\\trabajo\\SSMO\\EPH\\modulo_gestion\\1.analisis_problemas_de_salud\\egresos-hospitalarios\\calculo_metricas.py'>

In [2]:
HOSPITAL_A_ANALIZAR = 112103  # Este es el Torax


## Lectura de archivos

- Se leen todos los archivos que estan en input/utf-8. Los archivos estan separados por ;
- Se dejan solamente los egresos de los diagnosticos que **SI** tenga el Tórax
- Se cambia la codificacion de INTERV_Q de 2 (No, 1 es SI) a 0.
- Se cambia la codificacion de CONDICION_EGRESO (orig 1 = Vivo, 2 = Fallecido) de 1 -> 0 y 2 -> 1
- Se cambia la codificacion de GLOSA_REGION_RESIDENCIA {
                        "Del Libertador B. O'Higgins": "del Libertador General Bernardo O'Higgins",
                        "De Aisén del Gral. C. Ibáñez del Campo": "Aysén del General Carlos Ibáñez del Campo",
                    }

In [3]:
df = fr.leer_archivos(HOSPITAL_A_ANALIZAR)

# Analisis de Egresos Hospitalarios

Se calculan las siguientes metricas:

- Número de Egresos (n_egresos)
- Días de Estada promedio (dias_estada_promedio)
- Cantidad de Intervenciones Quirúrgicas (n_int_q)
- Cantidad de Muertes (n_muertos)

Todas estas metricas estan calculadas a nivel de Año, Establecimiento de Salud y Diagnóstico. Por
ejemplo:

***"En el año 2019, el Hospital San José (código XXXXXXX) tuvo XX egresos, con XX dias estada_promedio,
con XX intervenciones_quirúrgicas y XX muertos para el diagnóstico A24.4."***

Este análisis se realizó entre 2001 y 2020 para todos los hospitales/clínicas de Chile.

In [4]:
AGRUPACION = [
    "ANO_EGRESO",
    "ESTABLECIMIENTO_SALUD",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "DIAG1",
]

metricas = aux.obtener_metricas_egresos(df, AGRUPACION).collect(streaming=True)


# Analisis de Estratos

In [5]:
variables_a_rankear = ["n_egresos"]
subgrupo_del_ranking = ["ANO_EGRESO", "DIAG1"]
unir_tablas_por = AGRUPACION

In [6]:
dict_estratos = aux.obtener_diccionario_estratos(df, HOSPITAL_A_ANALIZAR)

In [7]:
dict_resultados_estratos = aux.obtener_resumen_por_estratos(
    metricas, dict_estratos, variables_a_rankear, subgrupo_del_ranking
)

resultado = reduce(aux.left_join_consecutivo, dict_resultados_estratos.values())

# Analisis Sociodemográfico

In [106]:
desglose_sociodemografico = [
    "ANO_EGRESO",
    "REGION_RESIDENCIA",
    "GLOSA_REGION_RESIDENCIA",
    "region_pais",
    "COMUNA_RESIDENCIA",
    "GLOSA_COMUNA_RESIDENCIA",
    "comuna_region_pais",
    "DIAG1",
]

variables_a_analizar = [
    "DIAG1",
    "SEXO",
    "PUEBLO_ORIGINARIO",
    "GLOSA_PAIS_ORIGEN",
    "PREVISION",
    "BENEFICIARIO",
]

In [50]:
with pl.StringCache():
    df_socio = (
        pl.read_csv("input/utf-8/*.csv", separator=";", dtypes=fr.DICT_VARIABLES)
        .lazy()
        .filter(pl.col("ESTABLECIMIENTO_SALUD") == 112103)
        .with_columns(
            (("Región " + pl.col("GLOSA_REGION_RESIDENCIA") + ", Chile").alias("region_pais"))
        )
        .with_columns(
            (pl.col("GLOSA_COMUNA_RESIDENCIA") + ", " + pl.col("region_pais")).alias(
                "comuna_region_pais"
            )
        )
    )

In [116]:
dict_resultados = {}

for variable in variables_a_analizar:
    agrupacion_total = desglose_sociodemografico.copy()
    if variable != "DIAG1":
        agrupacion_total += [variable]

    resultado = (
        df_socio.groupby(agrupacion_total)
        .agg(pl.col(variable).count().alias("conteo"))
        .sort(["ANO_EGRESO", "DIAG1", "conteo"], descending=True)
        .with_columns(
            pl.concat_str(pl.col(desglose_sociodemografico), separator="-").alias("llave_id")
        )
    ).collect()

    dict_resultados[variable] = resultado

In [119]:
import xlsxwriter

with xlsxwriter.Workbook('prueba.xlsx') as f:
    for var, df in dict_resultados.items():
        df.write_excel(workbook=f, worksheet=var)